# Denoising MRI Images

Commonly used filters 
* Bilateral 
* Non Local means
* Block matching 3D filtering (BM3D)

In [ ]:
from skimage import img_as_float, io
from skimage.metrics import peak_signal_noise_ratio
from matplotlib import pyplot as plt
from scipy import ndimage as nd

noisy_img = img_as_float(io.imread(""))
ref_img = img_as_float(io.imread(""))

from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral, denoise_wavelet, estimate_sigma)

denoise_TV = denoise_tv_chambolle(peak_signal_noise_ratio, weight=0.3, channel_axis=False)
noise_psnr = peak_signal_noise_ratio(ref_img, noisy_img)
TV_cleaned_psnr = peak_signal_noise_ratio(ref_img, denoise_TV)

print("PSNR of input image = ", noise_psnr)
print("PSNR of cleaned image = ", TV_cleaned_psnr)
plt.save("images/mri/tv_smoothed.tif", denoise_TV, cmap='gray')


Wavelet Filter -> same code as above

In [ ]:
walvelet_smoothed = denoise_wavelet(noisy_img, channel_axis=False, 
                                    method='BayesShrink', mode='soft', rescale_sigme=True)

noise_psnr = peak_signal_noise_ratio(ref_img, noisy_img)
walvelet_cleaned_psnr = peak_signal_noise_ratio(ref_img, walvelet_smoothed)

print("PSNR of input image = ", noise_psnr)
print("PSNR of cleaned image = ", walvelet_cleaned_psnr)

Anisotropic Diffusion

In [ ]:
from medpy.filter.smoothing import anisotropic_diffusion

img_aniso_filtered = anisotropic_diffusion(noisy_img, niter=50, kappa=50, gamma=0.2, option=2)

noise_psnr = peak_signal_noise_ratio(ref_img, noisy_img)
aniso_cleaned_psnr = peak_signal_noise_ratio(ref_img, img_aniso_filtered)

print("PSNR of input image = ", noise_psnr)
print("PSNR of cleaned image = ", aniso_cleaned_psnr)

plt.imshow(img_aniso_filtered, cmap='gray')
plt.imsave("images/mri/anisotropic_denoised.tif", img_aniso_filtered, cmap='gray')

Non Local Means filter

In [ ]:
from skimage import img_as_ubyte

sigma_est = np.mean(estimate_sigma(noisy_img, channel_axis=False))

NLM_skimg_denoise_img = denoise_nl_means(noisy_img, h=1.15 * sigma_est, fast_mode=True,
                                         patch_size=9, patch_distance=5, channel_axis=True)

noise_psnr = peak_signal_noise_ratio(ref_img, noisy_img)
NLM_skimg_cleaned_psnr = peak_signal_noise_ratio(ref_img, NLM_skimg_denoise_img)

print("PSNR of input noisy image = ", noise_psnr)
print("PSNR of cleaned image = ", NLM_skimg_cleaned_psnr)

denoise_img_as_8byte = img_as_ubyte(NLM_skimg_denoise_img)

plt.imsave("images/mri/NLM_skimage_denoised.tif", denoise_img_as_8byte, cmap='gray')


OpenCv implementation

In [ ]:
import numpy as np 
import cv2

noise_psnr = peak_signal_noise_ratio(ref_img, noisy_img)
NLM_CV2_denoise_img = cv2.fastNlMeansDenoising(noisy_img, None, 3, 7, 21)

plt.imsave("images/mri/NLM_CV2_denoised.tif", NLM_CV2_denoise_img, cmap='gray')
plt.imshow("images/mri/NLM_CV2_denoised.tif", NLM_CV2_denoise_img)


Block matching 3D filtering (BM3D)

In [ ]:
import bm3d

BM3D_denoised_img = bm3d.bm3d(noisy_img, sigma_psd=0.2, stage_arg=bm3d.BM3DStages)

noise_psnr = peak_signal_noise_ratio(ref_img, noisy_img)
BM3D_cleaned_psnr= peak_signal_noise_ratio(ref_img, BM3D_denoised_img)

print("PSNR of noisy input image = ", noise_psnr)
print("PSNR of cleaned image = ", BM3D_cleaned_psnr)

plt.imshow(BM3D_denoised_img, cmap='gray')
plt.imsave("images/mri/BM3D_denoised.tif", BM3D_denoised_img, cmap='gray')